In [1]:
# Dependencies
import json
import requests
import pandas as pd
import numpy as np
import os
from datetime import datetime
from api_keys import DarkSkyKey

In [2]:
# read taxi rides file
file = "Resources/Taxi_Trips_Random_Sample.csv"
df = pd.read_csv(file, header=None)
df.columns = ["Trip ID", "Taxi ID","Trip Start Timestamp", "Trip End Timestamp","Trip Seconds",
             "Trip Miles", "Pickup Census Tract", "Dropoff Census Tract", "Pickup Community Area", 
             "Dropoff Community Area", "Fare", "Tips", "Tolls", "Extras", "Trip Total", "Payment Type", 
             "Company", "Pickup Centroid Latitude", "Pickup Centroid Longitude", "Pickup Centroid Location", 
             "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Dropoff Centroid Location","Community Areas"]
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,11/16/2013 01:45:00 AM,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,09/28/2016 04:45:00 PM,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,12/10/2013 07:15:00 AM,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,02/27/2014 09:00:00 AM,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,10/31/2016 04:15:00 PM,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [3]:
# convert timestamp format for api calls
newdatestart_list = []

for date in df['Trip Start Timestamp']:
    date_obj = datetime.strptime(date, '%m/%d/%Y %H:%M:%S %p')
    new_date = date_obj.strftime('%Y-%m-%dT%H:%M:%S')
    newdatestart_list.append(new_date)

df['Trip Start Timestamp'] = newdatestart_list
df.head()


,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,bc6f08423e9f194cb63a0d5ec6c6a2dea001e914,07a82232d832859a7dcb1c40846bb3e21acb2c84755ef6...,2013-11-16T01:45:00,11/16/2013 01:45:00 AM,480.0,0.15,1.703108e+10,1.703108e+10,8.0,8.0,...,8.05,Cash,NaN,41.907413,-87.640902,POINT (-87.6409015248 41.9074128162),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),37.0
1,b2e0ede2b0d907551beda4e8b4b83c143010f63d,74605d6aa0c8ba08190a5824f79ef27172c59671b00582...,2016-09-28T04:45:00,09/28/2016 04:45:00 PM,420.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,8.75,Credit Card,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38.0
2,c2155aedc41a27673cd498d0fea754a18132d7f9,4243ab34318764eb5ca51054af503c610b50579594cef6...,2013-12-10T07:15:00,12/10/2013 07:30:00 AM,720.0,1.80,1.703128e+10,1.703108e+10,28.0,8.0,...,8.44,Cash,NaN,41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),29.0
3,c1e860354f05150021e1bdeb7696314ca3e15608,65e345a16c02e0408409d6d71a7a5cb09449e5c84c16d0...,2014-02-27T09:00:00,02/27/2014 09:45:00 AM,2700.0,0.00,1.703108e+10,1.703198e+10,8.0,76.0,...,50.70,Credit Card,Choice Taxi Association,41.902788,-87.626146,POINT (-87.6261455896 41.9027880476),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),37.0
4,bdff93a935333338e6f2f04f778ad7713a36c9ad,8ca267f2246008a052c14ce8c81001149d86916ab274d3...,2016-10-31T04:15:00,10/31/2016 04:15:00 PM,660.0,3.00,NaN,NaN,32.0,35.0,...,13.50,Credit Card,NaN,41.878866,-87.625192,POINT (-87.6251921424 41.8788655841),41.835118,-87.618678,POINT (-87.6186777673 41.8351179863),38.0


In [4]:
# Save config information
url = "https://api.darksky.net/forecast/"
exception_count = 0

weather_df=pd.DataFrame()

# calculate begining of iteration from last line in csv file if existant
try:
    with open("Output/weather.csv", "r") as f:
        a = int(pd.read_csv("Output/weather.csv", header=None).tail(1)[0]+1)
except:
    a = 0

n = a
        
# Build query URL and DataFrame
for i in range(a, len(df)):
    #get info from taxi ride file
    lat = df['Pickup Centroid Latitude'][i]
    lng = df['Pickup Centroid Longitude'][i]
    time = df['Trip Start Timestamp'][i]

    #make API calls with this info and save to DataFrame
    try:
        query_url = url + DarkSkyKey + "/" + str(lat) +"," + str(lng) + "," + time 
        weather_response = requests.get(query_url)
        weather_json = weather_response.json()['currently'] 
        weather_df.loc[i,'Trip ID'] = df['Trip ID'][i]
        weather_df.loc[i,'time'] = time
        weather_df.loc[i,'summary'] = weather_json['summary']
        weather_df.loc[i,'precipIntensity'] = weather_json['precipIntensity']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'temperature'] = weather_json['temperature']
        weather_df.loc[i,'apparentTemperature'] = weather_json['apparentTemperature']
        weather_df.loc[i,'windSpeed'] = weather_json['windSpeed']
        weather_df.loc[i,'precipProbability'] = weather_json['precipProbability']
        weather_df.loc[i,'icon'] = weather_json['icon']
    
    except:
        if i-n <= 1: exception_count += 1 # count consecutive failed calls
        else: exception_count = 0
            
        n = i
        
        if exception_count >= 5: 
            print(f"Request no {i} failed, 5 consecutive requests have failed, ending itteration")
            break
        else: 
            print(f"Request no {i} failed, skipping to next line")
weather_df

From cffi callback <function _verify_callback at 0x0000021AAAE18C80>:
Traceback (most recent call last):
  File "C:\Users\porphan\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


Request no 2154 failed, skipping to next line
Request no 2223 failed, skipping to next line
Request no 2344 failed, skipping to next line
Request no 2618 failed, skipping to next line
Request no 2652 failed, skipping to next line
Request no 2876 failed, skipping to next line
Request no 2901 failed, skipping to next line
Request no 2916 failed, skipping to next line
Request no 3017 failed, skipping to next line
Request no 3018 failed, skipping to next line
Request no 3019 failed, skipping to next line
Request no 3020 failed, skipping to next line
Request no 3021 failed, skipping to next line
Request no 3022 failed, 5 consecutive requests have failed, ending itteration


,Trip ID,time,summary,precipIntensity,precipProbability,temperature,apparentTemperature,windSpeed,icon
2009,b2d3ee7d8d364bd3f4d09135c09b4b49e6a1c122,2014-11-07T09:00:00,Partly Cloudy,0.0000,0.00,38.20,35.30,3.94,partly-cloudy-day
2010,b1ab8e38c1fd33c47344967e8c973dcab527c0d1,2014-03-08T02:15:00,Overcast,0.0000,0.00,33.36,28.40,5.32,cloudy
2011,c2397711243ea4e1bfbc2cda3753c4b1e3e72661,2014-10-10T09:45:00,Partly Cloudy,0.0000,0.00,52.59,52.59,2.78,partly-cloudy-day
2012,b92947706d94e83cdd9bffdf9fc42fa337aefed5,2017-03-27T06:30:00,Overcast,0.0000,0.00,47.41,47.41,1.00,cloudy
2013,b1238538e96a1b7b05802c69a198102e2bb92e03,2016-11-06T10:45:00,Clear,0.0000,0.00,63.79,63.79,1.80,clear-day
2014,bef3b2b72661b923b218a614415a308611fd6ab5,2013-08-05T08:00:00,Partly Cloudy,0.0000,0.00,65.79,65.79,3.10,partly-cloudy-day
2015,c40d71bc7b339ce7d244ed30cbcb44b03125601d,2014-01-04T11:30:00,Overcast,0.0000,0.00,29.81,21.83,8.56,cloudy
2016,c0614124f66f8684d4ab4df57895cdca750a7dbb,2015-03-11T02:30:00,Foggy,0.0000,0.00,38.90,38.90,1.00,fog
2017,ba6893932391e9d2e8f407855e0e0f135f11f7d4,2014-07-18T11:15:00,Partly Cloudy,0.0000,0.00,75.50,75.50,1.06,partly-cloudy-day
2018,bf150b0106beeddc4973cae0f5e92904f5bbd953,2014-10-11T02:15:00,Clear,0.0000,0.00,43.33,43.33,0.44,clear-night


In [ ]:
# Save results in csv output file (append if file already exists)
try: os.mkdir('Output')
except: pass

try:
    with open("Output/weather.csv", "a") as f:
        weather_df.to_csv(f, header=False)
except:
    weather_df.to_csv("Output/weather.csv")